En este archivo, voy a desarrollar la función que se me pide en el quinto endpoint, que es la que se muestra a continuación:

![alt text](../img/endpoint_5.png "Title")

# Importaciones necesarias para trabajar.

In [2]:
import pandas as pd

In [3]:
reseñas = pd.read_parquet("../Datasets/reviews_con_puntaje.parquet")
reseñas.head(3)

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,puntaje
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,2
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,2
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,2


In [4]:
juegos = pd.read_parquet("../Datasets/steam_games_complete.parquet")
juegos.head(3)

,item_id,item_name,developer,genres,tags,specs,release_date,price
88310,761140,Lost Summoner Kitty,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]","[Strategy, Action, Indie, Casual, Simulation]",[Single-player],2018-01-04,4.99
88311,643980,Ironbound,Secret Level SRL,"[Free to Play, Indie, RPG, Strategy]","[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",2018-01-04,0.00
88312,670290,Real Pool 3D - Poolians,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]","[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",2017-07-24,0.00


## Segunda parte, creando endpoint.

Se me ocurren varias maneras de hacerlo, voy a ir por la que me resulta más sencilla y menos rebuscada.
A partir de la tabla de juegos, voy a filtrar aquellos juegos con el desarrollador ingresado...

Lo primero que tengo que hacer es determinar que columnas me sirven para realizar el endpoint. Analicemos...

In [11]:
print(juegos.columns)
print()
print(reseñas.columns)

Index(['item_id', 'item_name', 'developer', 'genres', 'tags', 'specs',
       'release_date', 'price'],
      dtype='object')

Index(['funny', 'posted', 'last_edited', 'item_id', 'helpful', 'recommend',
       'review', 'user_id', 'puntaje'],
      dtype='object')


De la tabla "juegos" me sirven las columnas: 'item_id' y 'developer', ya que con "item_id" puedo relacionar ambas tablas y "developer" va a ser lo que se ingresa a la función, el resto no me parecen relevantes para el endpoint.

De la tabla "reseñas", me sirven las columnas: 'item_id' y 'puntaje', ya que con "item_id" puedo relacionar ambas tablas, y con "puntaje" puedo determinar la cantidad de reseñas positivas o negativas que tiene el desarrollador historicamente. 

In [12]:
# Procedo a eliminar entonces las columnas que no me sirven para optimizar el rendimiento de la función.
for column in juegos.columns:
    if column not in ['item_id', 'developer']:
        juegos.drop(column, axis=1, inplace=True)
print(juegos.columns)

for column in reseñas.columns:
    if column not in ['item_id', 'puntaje']:
        reseñas.drop(column, axis=1, inplace=True)
print(reseñas.columns)

Index(['item_id', 'developer'], dtype='object')
Index(['item_id', 'puntaje'], dtype='object')


Una vez eliminadas, hechemos un vistazo a cómo quedaron las tablas

In [13]:
juegos

,item_id,developer
88310,761140,Kotoshiro
88311,643980,Secret Level SRL
88312,670290,Poolians.com
88313,767400,彼岸领域
88315,772540,Trickjump Games Ltd
...,...,...
120439,745400,Bidoniera Games
120440,773640,"Nikita ""Ghost_RUS"""
120441,733530,Sacada
120442,610660,Laush Dmitriy Sergeevich


In [14]:
reseñas

,item_id,puntaje
0,1250,2
1,22200,2
2,43110,2
3,251610,2
4,227300,2
...,...,...
59300,70,2
59301,362890,2
59302,273110,2
59303,730,2


Una vez eliminadas las columnas, tengo que saber que desarrolladores tengo disponibles para usar de ejemplo, así que usando el método .unique() lo puedo averiguar

In [8]:
lista_desarrolladores = list(juegos["developer"].unique())
lista_desarrolladores

['Kotoshiro',
 'Secret Level SRL',
 'Poolians.com',
 '彼岸领域',
 'Trickjump Games Ltd',
 'Poppermost Productions',
 'Stegalosaurus Game Development',
 'Copperpick Studio',
 'Ghulam Jewel',
 'Apillo',
 'Tero Lunkka',
 'FrozenPepper',
 'Casey Labrack',
 'Stainless Games Ltd',
 'Valve',
 'dev4play',
 'ETGgames',
 'lalalaZero,Urbanoff',
 'Strategy First',
 'Outerlight Ltd.',
 'Ultraint',
 'TPM.CO SOFT WORKS',
 'Wonderbox Games',
 'Mad Unicorn Games',
 'GlyphX Games',
 'Introversion Software',
 'Facepunch Studios',
 'SimBin',
 'Unknown Worlds Entertainment',
 'CINEMAX, s.r.o.',
 'Sick Puppies',
 'Deepred',
 'Funcom',
 'Bugbear Entertainment',
 'CAPCOM Co., Ltd.',
 'FireFly Studios,Firaxis Games',
 'Arkane Studios',
 'Firaxis Games',
 'PopTop',
 'Firaxis Games,Feral Interactive (Mac)',
 'MicroProse Software, Inc',
 'Crystal Dynamics,Feral Interactive (Mac)',
 'Telltale Games',
 'id Software',
 'Gray Matter Studios',
 'Ritual Entertainment',
 'Terminal Reality',
 'Rogue Entertainment',
 'Metamor

Primer prototipo del endpoint

In [15]:
def analisis_reseñas_desarrollador(desarrollador):
    """
    En este primer prototipo del endpoint, se va a ingresar a la función un nombre de un desarrollador, y va a devolver todos los juegos de ese desarrollador.
    """
    try:
        juegos[juegos["developer"] == desarrollador]
        juegos_filtrados_por_desarrollador = juegos[juegos["developer"] == desarrollador]
        return juegos_filtrados_por_desarrollador
    except Exception as e:
        return e

In [16]:
analisis_reseñas_desarrollador("Valve") # La salida es la esperada

,item_id,developer
88338,70,Valve
88752,630,Valve
88942,620,Valve
89355,730,Valve
101177,772170,Valve
106176,629330,Valve
119752,300,Valve
119839,550,Valve
120064,500,Valve
120119,400,Valve


Me gustaría que tenga también cómo salida la cantidad de juegos lanzados por la desarrolladora. Lo agrego a continuación

Segundo prototipo del endpoint

In [23]:
def analisis_reseñas_desarrollador(desarrollador):
    """
    En este segundo prototipo del endpoint, se va a ingresar a la función un nombre de un desarrollador, y va a devolver la cantidad de juegos de ese desarrollador.
    """
    try:
        juegos[juegos["developer"] == desarrollador]
        juegos_filtrados_por_desarrollador = juegos[juegos["developer"] == desarrollador]
        print(f"La desarrolladora de juegos {desarrollador} tiene {len(juegos_filtrados_por_desarrollador)} juegos")
        return juegos_filtrados_por_desarrollador
    except Exception as e:
        return e

In [24]:
analisis_reseñas_desarrollador("Valve") # Perfecto

La desarrolladora de juegos Valve tiene 24 juegos


,item_id,developer
88338,70,Valve
88752,630,Valve
88942,620,Valve
89355,730,Valve
101177,772170,Valve
106176,629330,Valve
119752,300,Valve
119839,550,Valve
120064,500,Valve
120119,400,Valve


Ahora lo que me gustaría es que a partir de esos id que ofrece la columna "item_id" se encuentren las reseñas correspondientes. Es decir, cada id es un juego, y puede ser que ese juego tenga más de una reseña. El tema es que no todas las reseñas corresponden a los juegos de una desarrolladora en concreto, entonces tengo que filtrar las reseñas, dejando sólo aquellas que hayan sido a juegos de la desarrolladora ingresada en la función

Tercer prototipo del endpoint 

In [48]:
def analisis_reseñas_desarrollador(desarrollador):
    """
    En este tercer prototipo del endpoint, se va a ingresar a la función un nombre de un desarrollador, y va a devolver la cantidad de juegos de ese desarrollador, y las reseñas de todos los juegos de ese desarrollador.
    """
    try:
        juegos[juegos["developer"] == desarrollador]

        juegos_filtrados_por_desarrollador = juegos[juegos["developer"] == desarrollador]

        id_juegos_desarrollador = juegos_filtrados_por_desarrollador["item_id"].values

        reseñas_juegos_desarrolladora = reseñas[reseñas["item_id"].isin(id_juegos_desarrollador)]

        print(f"La desarrolladora de juegos {desarrollador} tiene {len(juegos_filtrados_por_desarrollador)} juegos, \nlos cuáles cuentan con {len(reseñas_juegos_desarrolladora)} reseñas")

        return reseñas_juegos_desarrolladora
    except Exception as e:
        return e

In [49]:
analisis_reseñas_desarrollador("Valve") # La salida es la esperada

La desarrolladora de juegos Valve tiene 24 juegos, 
los cuáles cuentan con 9557 reseñas


,item_id,puntaje
20,730,1
24,550,2
26,220,0
27,730,1
28,730,2
...,...,...
59288,440,1
59296,730,1
59300,70,2
59303,730,2


Para mi función, sólo me piden las reseñas correspondientes a las negativas y a las positivas, las cuáles corresponden al puntaje 0 y 2 respectivamente. Sin embargo, yo voy a tener en consideración las neutras, y la salida se va a ver algo así:

{Desarrollador : [Reseñas negativas = x, reseñas neutras = y, reseñas positivas = z]}

o así:

{Desarrollador : {Cantidad de reseñas : [Negativas = x, Neutras = y, Positivas = z]}}

Para ello, primero tengo que determinar la cantidad de reseñas, tanto las negativas cómo las neutras cómo las positivas

Cuarto prototipo del endpoint

In [60]:
def analisis_reseñas_desarrollador(desarrollador):
    """
    En este cuarto prototipo del endpoint, se va a ingresar a la función un nombre de un desarrollador, y va a devolver la cantidad de juegos de ese desarrollador, las reseñas de todos los juegos de ese desarrollador, y la cantidad de reseñas correspondientes a negativas, neutras y positivas.
    """
    try:
        juegos[juegos["developer"] == desarrollador]

        juegos_filtrados_por_desarrollador = juegos[juegos["developer"] == desarrollador]

        id_juegos_desarrollador = juegos_filtrados_por_desarrollador["item_id"].values

        reseñas_juegos_desarrolladora = reseñas[reseñas["item_id"].isin(id_juegos_desarrollador)]

        cant_reseñas_negativas = len(reseñas_juegos_desarrolladora[reseñas_juegos_desarrolladora["puntaje"] == 0])

        cant_reseñas_neutras = len(reseñas_juegos_desarrolladora[reseñas_juegos_desarrolladora["puntaje"] == 1])

        cant_reseñas_positivas = len(reseñas_juegos_desarrolladora[reseñas_juegos_desarrolladora["puntaje"] == 2])

        print(f"La desarrolladora de juegos {desarrollador} tiene {len(juegos_filtrados_por_desarrollador)} juegos, \nlos cuáles cuentan con {len(reseñas_juegos_desarrolladora)} reseñas. \nDe las cuáles {cant_reseñas_positivas} son positivas, \n{cant_reseñas_neutras} son neutras y, \n{cant_reseñas_negativas} son negativas")
        
        return 
    except Exception as e:
        return e
reseñas_Valve = analisis_reseñas_desarrollador("Valve")
reseñas_Valve

La desarrolladora de juegos Valve tiene 24 juegos, 
los cuáles cuentan con 9557 reseñas. 
De las cuáles 6188 son positivas, 
2189 son neutras y, 
1180 son negativas


Ahora le tengo que agregar el diccionario de salida, todo ese texto hermoso que escribí va a ser transformado en un diccionario.
Bueno, no todo, sino a partir del punto y a parte, a partir de "De las cuales..."

Quinto prototipo del endpoint

In [66]:
def analisis_reseñas_desarrollador(desarrollador):
    """
    En este quinto prototipo del endpoint, se va a ingresar a la función un nombre de un desarrollador, y va a devolver la cantidad de juegos de ese desarrollador, las reseñas de todos los juegos de ese desarrollador, y mediante un diccionario, la cantidad de reseñas correspondientes a negativas, neutras y positivas.
    """
    try:
        diccionario_salida = {}
        juegos[juegos["developer"] == desarrollador]

        juegos_filtrados_por_desarrollador = juegos[juegos["developer"] == desarrollador]

        id_juegos_desarrollador = juegos_filtrados_por_desarrollador["item_id"].values

        reseñas_juegos_desarrolladora = reseñas[reseñas["item_id"].isin(id_juegos_desarrollador)]

        cant_reseñas_negativas = len(reseñas_juegos_desarrolladora[reseñas_juegos_desarrolladora["puntaje"] == 0])

        cant_reseñas_neutras = len(reseñas_juegos_desarrolladora[reseñas_juegos_desarrolladora["puntaje"] == 1])

        cant_reseñas_positivas = len(reseñas_juegos_desarrolladora[reseñas_juegos_desarrolladora["puntaje"] == 2])

        lista_reseñas = [f"Negativas = {cant_reseñas_negativas}",       f"Neutras = {cant_reseñas_neutras}", 
            f"Positivas = {cant_reseñas_positivas}"]
        
        diccionario_salida_interno = {}

        diccionario_salida_interno["Cantidad de reseñas"] = lista_reseñas

        diccionario_salida[desarrollador] = diccionario_salida_interno

        print(f"La desarrolladora de juegos {desarrollador} tiene {len(juegos_filtrados_por_desarrollador)} juegos, \nlos cuáles cuentan con {len(reseñas_juegos_desarrolladora)} reseñas.")

        return diccionario_salida
    except Exception as e:
        return e

reseñas_Valve = analisis_reseñas_desarrollador("Valve")
reseñas_Valve

La desarrolladora de juegos Valve tiene 24 juegos, 
los cuáles cuentan con 9557 reseñas.


{'Valve': {'Cantidad de reseñas': ['Negativas = 1180',
   'Neutras = 2189',
   'Positivas = 6188']}}

Y cómo el print no se va a ver en el despliegue de la API, se me ocurrió otra forma de hacerlo